# Summation circuit
---
Definition of functions. See [arXiv:quant-ph/0008033](https://arxiv.org/abs/quant-ph/0008033).

*Author*: Sebastián V. Romero ([sebastian.vidal@rai.usc.es](mailto:sebastian.vidal@rai.usc.es)).

## General imports

In [1]:
import numpy as np

from qiskit import QuantumCircuit, execute, Aer
from qiskit.circuit.library import QFT

## Some previous functions

In [2]:
def intToBinary(value: str, nBits: int, reverse: bool = True):
    
    value = int(value)
    if value != 0:
        bound = int(np.ceil(np.log2(value)))
        if nBits < bound: nBits = bound
    
    binaryFormat = '{0:0' + str(nBits) + 'b}'
    binary = binaryFormat.format(value)
    
    if reverse: return binary[ : : -1]
    else: return binary

In [3]:
def binaryToInt(value: str, reverse: bool = True):
    
    if reverse: return int(value[ : : -1], 2)
    else: return int(value, 2)

In [4]:
def getState(circuit, nBits: int, precision: int = 10, fancyPrint: bool = False):
    
    # Let's simulate our circuit in order to get the final state vector
    svsim = Aer.get_backend('statevector_simulator', device='GPU')

    # Do the simulation, return the result and get the state vector
    result = execute(circuit, svsim).result().get_statevector()
    
    if int(precision) < 8: precision = 8
    
    result = np.asarray(result).round(precision)
    
    # Returns non-zero values
    stateIndices = result.nonzero()[0]
    
#     if nBits < int(np.log2(len(circuit))): nBits = int(np.log2(len(circuit)))
    states = np.empty(0)
    for index in stateIndices:
        binaryState = intToBinary(index, nBits)
        states = np.append(states , [result[index], binaryState])
    
    if fancyPrint:
        string = ''
        for index in range(0, len(states), 2):
            coef, bstate = states[index], states[index + 1]
            string = string + f'{coef} |{bstate}>   '
        print(string)
    
    return states

In [6]:
def summation(circuit, state1, state2):
    
    assert len(state1) == len(state2)
    n = len(state1)
    
    if n == 0: return circuit
    for phase in range(n):
        for control in range(0, phase + 1):
            dif = phase - control
            circuit.cp(np.pi / pow(2, dif), state1[control], state2[phase])
            circuit.barrier()
    
    return circuit